# Logan Clark - College To NFL Trends (RB)

Predicting a players NFL Fantasy Football success by using College Football stats along with Random Forest Regression, and Neural Network Regression. 

**Data**

Evaluation Methodology: The current value metric I am using for each player is

Value = (AVG EP Season 1-3 + AVG PPG Season 1-3)/2 + (#Top-12 Finishes * 2)

My logic behind this is this first part averages a players Expected Points and Actual Points. And then I need the second part because I only have Season 1-3 stats for EP and PPG and how long a player performs at a high level is relevant to Dynasty Purposes.





Source: Pahowdy's Database

https://docs.google.com/spreadsheets/d/1D1D9rXo3BXX867qBju77VjZ9YIoAp-nXYFqSGDQgajU/edit#gid=224755041











In [ ]:
!pip install tensorflow-addons

import math
import time

import matplotlib.pyplot as plt
import numpy as np
from numpy.random import seed
import pandas as pd

import tensorflow as tf
import tensorflow_addons as tfa

import torch

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor

from sklearn import metrics

tf.keras.backend.set_floatx('float64')

!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py 

from helper_functions import create_tensorboard_callback, plot_loss_curves

     |████████████████████████████████| 1.1 MB 5.4 MB/s 
--2021-10-15 23:35:15--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-10-15 23:35:15 (46.9 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
def eval_model(model, X_test, y_test, hist):
    print("eval", model.evaluate(X_test, y_test))
    print("predict", model.predict(X_test[:10]))
    pd.DataFrame(hist.history).plot()
    plt.ylabel("loss")
    plt.xlabel("epochs")

metrics = [tf.metrics.mae, tfa.metrics.RSquare(dtype=tf.float32, y_shape=(1,))]
    

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Data Prep

In [ ]:
data = pd.read_csv('/content/gdrive/My Drive/pahowdy_cfb_rb_training.csv')
rookie_data = pd.read_csv('/content/gdrive/My Drive/pahowdy_cfb_rb_rookies.csv')

features = (data.columns)[4:98]

print(data.head())
print(rookie_data.head())

data = data.to_numpy()
rookie_data = rookie_data.to_numpy()
rookie_names = rookie_data[:,0]

data = np.delete(data, 0, axis=0)



                  Name  ... Year 1-3 Score + RB1
0                    0  ...              109.000
1  LaDainian Tomlinson  ...               38.865
2      Adrian Peterson  ...               27.075
3        Malcolm Perry  ...                2.615
4       Anthony Thomas  ...               13.165

[5 rows x 110 columns]
               Name              cfb_id    pfr_id  ... Burst Score   HaSS   WaSS
0      Najee Harris      najee-harris-1  HarrNa00  ...      118.56  88.24  99.04
1    Travis Etienne    travis-etienne-1  EtieTr00  ...      120.95  88.24  99.04
2  Javonte Williams  javonte-williams-1  WillJa10  ...      122.70  88.24  99.04
3       Trey Sermon       trey-sermon-1  SermTr00  ...      125.40  88.24  99.04
4    Michael Carter    michael-carter-5  CartMi02  ...      117.30  83.31  95.10

[5 rows x 97 columns]


In [ ]:
def encode_cfb_data(input_data):
  ConferenceDict = {'ACC':0, 'ACC (Atlantic)':1, 'ACC (Coastal)':2, 'American':3, 'American (East)':4, 'American (West)':5,
                  'Big 12':6, 'Big 12 (North)':7, 'Big 12 (South)':8, 'Big East':9, 'Big Ten':10, 'Big Ten (East)':11,
                  'Big Ten (Leaders)':12, 'Big Ten (Legends)':13, 'Big Ten (West)':14, 'CUSA': 15, 'CUSA (East)':16,
                  'CUSA (West)':17, 'Ind':18, 'MAC (East)':19, 'MAC (West)':20, 'MWC':21, 'MWC (Mountain)':22,
                  'MWC (West)':23, 'Pac-10':24, 'Pac-12 (North)':25, 'Pac-12 (South)':26, 'SEC (East)':27,
                  'SEC (West)':28, 'Sun Belt':29, 'Sun Belt (East)':30, 'Sun Belt (West)':31, 'WAC':31}

  DraftRoundDict = {'UDFA':9}

  DraftPositionDict = {'UDFA':260}

  LastYearDesignationDict = {'SR':0, 'JR':1, 'SO':2, 'FR':3}

  for player in input_data:
    player[4] = ConferenceDict[(player[4].replace("\xa0", " "))]
    if not (player[5].isdigit()):
      player[5] = DraftRoundDict[player[5]]
    if not (player[6].isdigit()):
      player[6] = DraftPositionDict[player[6]]
    player[7] = LastYearDesignationDict[player[7]]

  return input_data

In [ ]:
# Data Encoding
data = encode_cfb_data(data)
rookie_data = encode_cfb_data(rookie_data)

In [ ]:
# Split data into X and y

X = list()
y = list()

X_rookie = list()

for player in data:
  X.append(player[4:97])
  y.append(player[109])

for player in rookie_data:
  X_rookie.append(player[4:97])

X = np.array(X).astype(float) 
y = np.array(y).astype(float)

X_rookie = np.array(X_rookie).astype(float)

In [ ]:
# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = tf.convert_to_tensor(X_train)
X_test = tf.convert_to_tensor(X_test)
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

# Baseline Random Forest Model

In [ ]:
rfReg = RandomForestRegressor(n_estimators=100).fit(X_train, y_train)
rf_r_squared = rfReg.score(X_test, y_test)
rf_r_squared

0.4816280193526142

In [ ]:
#List features by importance

feature_importance = rfReg.feature_importances_

feature_numbered_list = list()
for i in range(len(feature_importance)):
  feature_numbered_list.append((feature_importance[i], features[i]))


sorted_features = sorted(feature_numbered_list, reverse=True)
for i in sorted_features:
  print(i)

(0.2282650131564943, 'DP')
(0.07392148739752619, 'Post Draft Percentile')
(0.06842169668285418, 'Age Draft Hit Rate AVG')
(0.0561764800121399, 'Post Draft Model')
(0.021131059702625916, 'Production Score')
(0.01926593575541904, 'Best.9')
(0.018994889094211244, 'YPTch')
(0.018370154807869507, 'Last.9')
(0.015464554041474248, 'Last.8')
(0.01363423062803265, 'First.2')
(0.013105298815064616, 'REC/g')
(0.01187023201815707, 'AVG.9')
(0.011240058431860045, 'vert')
(0.011161286181832401, 'AVG YD/G')
(0.010923097577898434, 'Burst Score')
(0.010736474738682422, 'First.11')
(0.010666269519175452, 'First.10')
(0.01054496192421695, 'Conference AVG')
(0.010530205457984549, 'AVG.11')
(0.010408594867187594, 'Best.10')
(0.010265683428600451, 'Rush College Dominator Rating')
(0.01002802348480508, '40')
(0.009476106524352833, 'Best')
(0.009259510277782735, 'First.9')
(0.008863647325788426, 'Last')
(0.008791630336810491, 'Best.8')
(0.008686374004620807, 'Last.11')
(0.00857162935786856, 'First.5')
(0.0084

# Baseline Neural Network

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

# Pre-processing layer (normalization)
data_normalization = Sequential([
                                 preprocessing.Normalization()
], name="data_normalization")

# Setup input shape
input_shape = 94

# Assemble model
inputs = layers.Input(shape=input_shape, name="input_layer")
x = data_normalization(inputs)
x = layers.Dense(100)(x)
x = layers.LeakyReLU()(x)
x = layers.Dense(10)(x)
x = layers.LeakyReLU()(x)
outputs = layers.Dense(1, name="output_layer")(x)
model_0 = tf.keras.Model(inputs, outputs)

model_0.compile(loss=tf.keras.losses.mae,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=metrics)

history_0 = model_0.fit(X_train, y_train, epochs=50, callbacks=[create_tensorboard_callback(dir_name="rb_trends",
                                                                                            experiment_name="model_0")])

Saving TensorBoard log files to: rb_trends/model_0/20211015-233541
Epoch 1/50
13/13 [==============================] - 4s 28ms/step - loss: 241.0211 - mean_absolute_error: 241.0211 - r_square: -3069.6189
Epoch 2/50
13/13 [==============================] - 0s 6ms/step - loss: 73.7887 - mean_absolute_error: 73.7887 - r_square: -225.4870
Epoch 3/50
13/13 [==============================] - 0s 6ms/step - loss: 23.7336 - mean_absolute_error: 23.7336 - r_square: -35.6652
Epoch 4/50
13/13 [==============================] - 0s 6ms/step - loss: 10.3041 - mean_absolute_error: 10.3041 - r_square: -4.4428
Epoch 5/50
13/13 [==============================] - 0s 4ms/step - loss: 6.4290 - mean_absolute_error: 6.4290 - r_square: -1.1175
Epoch 6/50
13/13 [==============================] - 0s 6ms/step - loss: 5.4096 - mean_absolute_error: 5.4096 - r_square: -0.4375
Epoch 7/50
13/13 [==============================] - 0s 4ms/step - loss: 4.5188 - mean_absolute_error: 4.5188 - r_square: -0.0638
Epoch 8/50
13

In [ ]:
model_0.evaluate(X_test, y_test)

4/4 [==============================] - 0s 5ms/step - loss: 5.2380 - mean_absolute_error: 5.2380 - r_square: 0.1892


[5.237961627469204, 5.237961647945734, 0.18920302391052246]

In [ ]:
model_1 = tf.keras.Sequential([
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(10),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(1),
])

model_1.compile(loss=tf.keras.losses.mae,
                        optimizer=tf.keras.optimizers.Adam(),
                        metrics=metrics)

history = model_1.fit(X_train, y_train, epochs=50)

Epoch 1/50
13/13 [==============================] - 1s 6ms/step - loss: 22.5015 - mean_absolute_error: 22.5015 - r_square: -30.1400
Epoch 2/50
13/13 [==============================] - 0s 6ms/step - loss: 4.6026 - mean_absolute_error: 4.6026 - r_square: -0.3036
Epoch 3/50
13/13 [==============================] - 0s 6ms/step - loss: 4.8253 - mean_absolute_error: 4.8253 - r_square: -0.3448
Epoch 4/50
13/13 [==============================] - 0s 5ms/step - loss: 3.9524 - mean_absolute_error: 3.9524 - r_square: 0.0378
Epoch 5/50
13/13 [==============================] - 0s 6ms/step - loss: 3.7563 - mean_absolute_error: 3.7563 - r_square: 0.0810
Epoch 6/50
13/13 [==============================] - 0s 6ms/step - loss: 3.3894 - mean_absolute_error: 3.3894 - r_square: 0.2258
Epoch 7/50
13/13 [==============================] - 0s 6ms/step - loss: 3.3450 - mean_absolute_error: 3.3450 - r_square: 0.2311
Epoch 8/50
13/13 [==============================] - 0s 6ms/step - loss: 3.4194 - mean_absolute_err

In [ ]:
model_1.evaluate(X_test, y_test)

4/4 [==============================] - 0s 5ms/step - loss: 3.4847 - mean_absolute_error: 3.4847 - r_square: 0.3847


[3.4847054528717947, 3.4847054597133265, 0.38470423221588135]

In [ ]:
def normalized_predictions_with_names(predictions):
  pred_sum = sum(predictions)

  normalized_preds = [(float(i)/pred_sum)*1000 for i in predictions]

  names_preds = dict()

  for i in range(len(normalized_preds)):
      if type(normalized_preds[i]) is np.ndarray:
        names_preds[rookie_names[i]] = normalized_preds[i][0]
      else:
        names_preds[rookie_names[i]] = normalized_preds[i]   
          
  sorted_rookies = sorted(names_preds.items(), key=lambda x: x[1], reverse=True)

  # for i in sorted_rookies:
  #     print(i)
  return sorted_rookies

In [ ]:
rf_preds = rfReg.predict(X_rookie)
rf_preds_norm = normalized_predictions_with_names(rf_preds)

for i in rf_preds_norm:
  print(i)

('Travis Etienne', 50.04056769133225)
('Najee Harris', 48.26054062045303)
('Javonte Williams', 41.7851610215157)
('Trey Sermon', 32.46837502210108)
('Chuba Hubbard', 27.646429525202613)
('Jermar Jefferson', 26.122222087611114)
('Elijah Collins', 24.26125277174514)
('Spencer Brown', 24.25809233701681)
('Michael Carter', 23.657785318340924)
('Josh Johnson', 22.75390098603806)
('Pooka Williams', 22.355861789975403)
('Jaret Patterson', 22.32970041361311)
('CJ Verdell', 22.227161864649453)
('Rhamondre Stevenson', 20.820592830834745)
('Rakeem Boyd', 20.712435731242937)
('Kenny Gainwell', 20.337046317399988)
('Gerrid Doaks', 20.01099480126045)
('Jerome Ford', 19.939533860458734)
('Journey Brown', 19.505851983848768)
('Kennedy Brooks', 19.315699161027478)
('JaTarvious Whitlow', 18.941363225427303)
('Isaiah Bowser', 18.65077881012792)
('Elijah Mitchell', 18.340705047337163)
('Malik Davis', 18.293649685826438)
('Stevie Scott', 18.091557442920347)
('Jafar Armstrong', 17.049667460813673)
('B.J. Em

In [ ]:
model_1_preds = model_1.predict(X_rookie)
model_1_preds_norm = normalized_predictions_with_names(model_1.predict(X_rookie))

for i in model_1_preds_norm:
  print(i)

('Travis Etienne', 78.46763087794886)
('Najee Harris', 61.42822776298109)
('Javonte Williams', 60.278895349941635)
('Chuba Hubbard', 49.13325045329489)
('Trey Sermon', 46.26499659118862)
('Michael Carter', 43.21717790804869)
('Kenny Gainwell', 34.64632088421575)
('Rhamondre Stevenson', 28.952622492121296)
('Jermar Jefferson', 21.820814201582348)
('Kene Nwangwu', 19.892560861780222)
('Chase Hayden', 19.221797529808793)
('Jafar Armstrong', 19.122748363014942)
('Elijah Mitchell', 19.110561135918452)
('Jaret Patterson', 18.992575634102035)
('Alex Fontenot', 18.650033522694386)
('Elijah Collins', 18.638311448699124)
('Jerome Ford', 17.122264030103512)
('Gerrid Doaks', 17.014183476638152)
('Khalan Laborn', 16.850245984487643)
('Christopher Brown Jr.', 16.7672788979939)
('Khalil Herbert', 16.677281834925306)
('CJ Verdell', 16.015162789919533)
('Isaiah Bowser', 15.991686686632347)
('B.J. Emmons', 15.397437032017619)
('Amir Rasul', 15.22304834033653)
('Journey Brown', 15.034507305649202)
('Spen

# Model Comparison

* Similar R^2 values
* Random forest seems incremental with it's values while Model_2 is the opposite
* Ex: 0-->1 in model_2 drops 12 value points and 5-->6 drops 11 value points, largest drop in rf is 2-->3 with 4 value points.

# Training with Data that does not involve Draft Position to search for "Sleepers"

Theory: Players that's value increases the most when the data does not involve draft position are talented players who were not drafted high meaning they would have a higher % chance to breakout than the average player at their draft position.

In [ ]:
# Split data into X and y

X_sleeper = list()
y_sleeper = list()

X_sleeper_rookie = list()

for player in data:
  X_sleeper.append(player[7:97])
  y_sleeper.append(player[109])

for player in rookie_data:
  X_sleeper_rookie.append(player[7:98])

X_sleeper = np.array(X_sleeper).astype(float) 
y_sleeper = np.array(y_sleeper).astype(float)

X_sleeper_rookie = np.array(X_sleeper_rookie).astype(float)

In [ ]:
# Split data into train and test
X_train_sleeper, X_test_sleeper, y_train_sleeper, y_test_sleeper = train_test_split(X_sleeper, y_sleeper, test_size=0.2, random_state=42)

X_train_sleeper = tf.convert_to_tensor(X_train_sleeper)
X_test_sleeper = tf.convert_to_tensor(X_test_sleeper)
y_train_sleeper = tf.convert_to_tensor(y_train_sleeper)
y_test_sleeper = tf.convert_to_tensor(y_test_sleeper)

In [ ]:
#Train Random Forest
rf_sleeper = RandomForestRegressor(n_estimators=100).fit(X_train_sleeper, y_train_sleeper)
rf_r_squared_sleeper = rf_sleeper.score(X_test_sleeper, y_test_sleeper)
rf_r_squared_sleeper

0.4825651694573894

In [ ]:
#List features by importance

sleeper_features = features[3:]

feature_importance = rf_sleeper.feature_importances_

feature_numbered_list = list()
for i in range(len(feature_importance)):
  feature_numbered_list.append((feature_importance[i], sleeper_features[i]))


sorted_features = sorted(feature_numbered_list, reverse=True)
for i in sorted_features:
  print(i)

(0.16835972721323766, 'Age Draft Hit Rate AVG')
(0.13826333578853955, 'Post Draft Percentile')
(0.10617255657480637, 'Post Draft Model')
(0.0205057721954914, 'YPTch')
(0.018846704109902107, 'Production Score')
(0.017596129262462238, '40')
(0.01649753979863749, 'Best.9')
(0.015083536904080526, 'Rush College Dominator Rating')
(0.01458690078901565, 'First.11')
(0.01337572177369986, 'Last.9')
(0.013174630047916847, 'AVG.9')
(0.01297117188795197, 'First.5')
(0.012430546192012313, 'AVG YD/G')
(0.012291515987576438, 'First.9')
(0.011850858264276634, 'vert')
(0.011627485447130905, 'Last')
(0.011521472295617481, 'Conference AVG')
(0.011229000046348971, 'First.2')
(0.010561777402176882, 'Best')
(0.010244036715117253, 'Last.8')
(0.010201204917736184, 'Last.2')
(0.009484911387539872, 'Burst Score')
(0.009253094997618548, 'First.1')
(0.008970228607213235, 'Best.8')
(0.0088170481361934, 'AVG.8')
(0.008803464833865442, 'Touches')
(0.00871894065669429, 'First.8')
(0.008698414253127984, 'WaSS')
(0.008

In [ ]:
# Retrain model_1 on new data
model_1_sleeper = tf.keras.Sequential([
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(10),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(1),
])

model_1_sleeper.compile(loss=tf.keras.losses.mae,
                        optimizer=tf.keras.optimizers.Adam(),
                        metrics=metrics)

history = model_1_sleeper.fit(X_train_sleeper, y_train_sleeper, epochs=50)

Epoch 1/50
13/13 [==============================] - 1s 5ms/step - loss: 16.9198 - mean_absolute_error: 16.9198 - r_square: -17.6388
Epoch 2/50
13/13 [==============================] - 0s 5ms/step - loss: 4.3927 - mean_absolute_error: 4.3927 - r_square: -0.1622
Epoch 3/50
13/13 [==============================] - 0s 5ms/step - loss: 4.9020 - mean_absolute_error: 4.9020 - r_square: -0.2523
Epoch 4/50
13/13 [==============================] - 0s 5ms/step - loss: 3.7062 - mean_absolute_error: 3.7062 - r_square: 0.0476
Epoch 5/50
13/13 [==============================] - 0s 6ms/step - loss: 3.6524 - mean_absolute_error: 3.6524 - r_square: 0.0833
Epoch 6/50
13/13 [==============================] - 0s 6ms/step - loss: 3.6137 - mean_absolute_error: 3.6137 - r_square: 0.0710
Epoch 7/50
13/13 [==============================] - 0s 6ms/step - loss: 3.6752 - mean_absolute_error: 3.6752 - r_square: 0.1031
Epoch 8/50
13/13 [==============================] - 0s 6ms/step - loss: 3.6197 - mean_absolute_err

In [ ]:
model_1_sleeper.evaluate(X_test_sleeper, y_test_sleeper)

4/4 [==============================] - 0s 6ms/step - loss: 4.0979 - mean_absolute_error: 4.0979 - r_square: 0.2787


[4.097874953015016, 4.097874985132451, 0.278652548789978]

In [ ]:
rf_preds_sleeper = rf_sleeper.predict(X_sleeper_rookie)
rf_preds_norm_sleeper = normalized_predictions_with_names(rf_sleeper.predict(X_sleeper_rookie))

for i in rf_preds_norm_sleeper:
  print(i)


('Travis Etienne', 50.66179671692367)
('Najee Harris', 44.03738242915889)
('Trey Sermon', 36.44182849782475)
('Javonte Williams', 31.367606894073496)
('Pooka Williams', 29.05292168142207)
('Chuba Hubbard', 27.401441372109684)
('Jermar Jefferson', 26.63762748752736)
('Spencer Brown', 24.80562785942431)
('Michael Carter', 24.490397695596233)
('Rhamondre Stevenson', 23.555395847987793)
('Stevie Scott', 23.288688734135423)
('Elijah Collins', 23.213698565991162)
('Journey Brown', 22.348597056109952)
('CJ Verdell', 21.888137064382605)
('Josh Johnson', 21.227477076252093)
('Jaret Patterson', 20.78551013503999)
('Kennedy Brooks', 20.64638810364115)
('Kenny Gainwell', 20.298583025144104)
('JaTarvious Whitlow', 20.155219468397743)
('Isaiah Bowser', 19.69238422247581)
('B.J. Emmons', 19.315227788573868)
('Rakeem Boyd', 18.175784419214718)
('Jafar Armstrong', 18.11487611522426)
('Alex Fontenot', 17.704296461583816)
('Malik Davis', 17.601481887159814)
('Gerrid Doaks', 17.291680875776574)
('Javian H

In [ ]:
model_1_preds_sleeper = model_1_sleeper.predict(X_sleeper_rookie)
model_1_preds_norm_sleeper = normalized_predictions_with_names(model_1_sleeper.predict(X_sleeper_rookie))

for i in model_1_preds_norm_sleeper:
  print(i)

('Travis Etienne', 50.05005931926635)
('Najee Harris', 39.66014969197727)
('Chuba Hubbard', 35.011554590848824)
('Jaret Patterson', 32.74763969278868)
('Kenny Gainwell', 32.391239682671966)
('Michael Carter', 31.16580640888576)
('Jermar Jefferson', 28.872009348939596)
('Elijah Mitchell', 28.32350478403795)
('Javonte Williams', 27.155173402403452)
('Trey Sermon', 26.53995436730443)
('CJ Verdell', 25.59127540079733)
('Spencer Brown', 25.188483025723198)
('Pooka Williams', 24.644651683232567)
('Brenden Knox', 24.03989525931046)
('Kennedy Brooks', 22.707360310739578)
('Alex Fontenot', 21.60309088552693)
('Kylin Hill', 21.07926829874588)
('Stevie Scott', 20.8262576226045)
('Javian Hawkins', 20.632120851516195)
('Khalil Herbert', 20.30593727667866)
('Journey Brown', 19.957193004087053)
('Gerrid Doaks', 19.860102537906737)
('Josh Johnson', 19.686362891115614)
('Rakeem Boyd', 19.59954980615769)
('Keaontay Ingram', 18.335983077679032)
('JaTarvious Whitlow', 18.250756263198575)
('Brittain Brown'

In [ ]:
# Create list based off differences

diff_list = list()

for i in range(len(rf_preds)):
  diff_list.append(model_1_preds_sleeper[i]-model_1_preds[i])

diff_dict = dict()

for i in range(len(diff_list)):
    if type(diff_list[i]) is np.ndarray:
      diff_dict[rookie_names[i]] = diff_list[i][0]
    else:
      diff_dict[rookie_names[i]] = diff_list[i]   
        
sorted_diff = sorted(diff_dict.items(), key=lambda x: x[1], reverse=True)

for i in sorted_diff:
  print(i)

('Jaret Patterson', 5.574880499439096)
('Brenden Knox', 4.956723562508378)
('Javian Hawkins', 4.486462445348449)
('Pooka Williams', 4.450860039644764)
('Elijah Mitchell', 4.3557755609693425)
('Stevie Scott', 4.301929755370214)
('Spencer Brown', 4.241580486474188)
('Kylin Hill', 4.20675379770056)
('CJ Verdell', 4.152941236216016)
('Jermar Jefferson', 4.033844523899758)
('Rakeem Boyd', 4.032155753236822)
('Kennedy Brooks', 3.648196331719752)
('JaTarvious Whitlow', 3.4161538251369676)
('Keaontay Ingram', 3.241133548531776)
('Josh Johnson', 3.2160783035480303)
('Ty Chandler', 2.920620044292869)
('Journey Brown', 2.7967666656155776)
('Kenny Gainwell', 2.760622388122856)
('Alex Fontenot', 2.614982449330414)
('Khalil Herbert', 2.6060435594230795)
('Deon Jackson', 2.575641288936205)
('Demetric Felton', 2.4778846835730226)
('Brittain Brown', 2.466149000446706)
('J.D. King', 2.4418687517830753)
('Gerrid Doaks', 2.42676391156454)
('Chris Evans', 2.4255043205223883)
('Master Teague', 1.81625628998